In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request

from collections import Counter
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [3]:
train_data = pd.read_table('./dataset/ratings_train.txt')
test_data = pd.read_table('./dataset/ratings_test.txt')

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [5]:
train_data.head() # 1:긍정, 0:부정

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [7]:
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [8]:
train_data['document'].nunique(), train_data['label'].nunique()

(146182, 2)

In [9]:
train_data.drop_duplicates(subset=['document'], inplace=True)

In [10]:
len(train_data)

146183

In [11]:
train_data['label'].value_counts()

0    73342
1    72841
Name: label, dtype: int64

In [12]:
train_data.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [13]:
train_data = train_data.dropna(how='any')
train_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [14]:
test_data = test_data.dropna(how='any')
test_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [15]:
# https://www.unicode.org/charts/PDF/U3130.pdf
# https://www.unicode.org/charts/PDF/UAC00.pdf -> 가 ~ 힣
# pattern -> [^a-zA-Z0-9]
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data.head()

/var/folders/g5/jlzp3hhs653_qdq2kmf35jbh0000gn/T/ipykernel_72977/3559070244.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [16]:
train_data['document'] = train_data['document'].str.replace('^ +', '')
train_data['document'].replace('', np.nan, inplace=True)
train_data.isnull().sum()

/var/folders/g5/jlzp3hhs653_qdq2kmf35jbh0000gn/T/ipykernel_72977/1744609778.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', '')


id            0
document    789
label         0
dtype: int64

In [17]:
train_data = train_data.dropna(how = 'any')
train_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [18]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145393 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        145393 non-null  int64 
 1   document  145393 non-null  object
 2   label     145393 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.4+ MB


In [19]:
test_data.drop_duplicates(subset=['document'], inplace=True)
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['document'] = test_data['document'].str.replace('^ +', '')
test_data['document'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how = 'any')
test_data.isnull().sum()

/var/folders/g5/jlzp3hhs653_qdq2kmf35jbh0000gn/T/ipykernel_72977/849229089.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
/var/folders/g5/jlzp3hhs653_qdq2kmf35jbh0000gn/T/ipykernel_72977/849229089.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', '')


id          0
document    0
label       0
dtype: int64

In [20]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48852 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        48852 non-null  int64 
 1   document  48852 non-null  object
 2   label     48852 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [21]:
# 토큰화(Konlpy.Okt), 어간 추출, 표제어 추출, 불용어, 임베딩

In [22]:
okt = Okt()
okt.morphs('교도소 이야기구먼 솔직히 재미는 없다평점 조정', stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [23]:
X_train = []
stopwords = ['은','는','이','가','을','를','의','으로','에','와','한다','하다','한']

In [24]:
for data in tqdm(train_data['document']):
    tokenized_data = okt.morphs(data, stem=True)
    # 불용어처리 
    _data = []
    for word in tokenized_data:
        if not word in stopwords:
            _data.append(word)
    X_train.append(_data)

100%|█████████████████████████████████████████████| 145393/145393 [05:36<00:00, 431.49it/s]


In [25]:
X_train[:3]

[['아', '더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다']]

In [26]:
train_data['tokenzied'] = X_train
train_data.head()

,id,document,label,tokenzied
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"[아, 더빙, 진짜, 짜증나다, 목소리]"
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1,"[흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 조차, 가볍다, 않다]"
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[너, 무재, 밓었, 다그, 래서, 보다, 추천, 다]"
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,"[교도소, 이야기, 구먼, 솔직하다, 재미, 없다, 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,"[사이, 몬페, 그, 익살스럽다, 연기, 돋보이다, 영화, 스파이더맨, 에서, 늙다..."


In [29]:
Y_train = train_data['label'].values
Y_train

array([0, 1, 0, ..., 0, 1, 0])

In [30]:
X_test = []
for data in tqdm(test_data['document']):
    tokenized_data = okt.morphs(data, stem=True)
    # 불용어처리 
    _data = []
    for word in tokenized_data:
        if not word in stopwords:
            _data.append(word)
    X_test.append(_data)
test_data['tokenzied'] = X_test
Y_test = test_data['label'].values

test_data.head()

100%|███████████████████████████████████████████████| 48852/48852 [02:11<00:00, 372.07it/s]


,id,document,label,tokenzied
0,6270596,굳 ㅋ,1,"[굳다, ㅋ]"
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,"[뭐, 야, 평점, 들, 나쁘다, 않다, 점, 짜다, 리, 더, 더욱, 아니다]"
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,"[지루하다, 않다, 완전, 막장, 임, 돈, 주다, 보기, 에는]"
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,"[만, 아니다, 별, 다섯, 개, 주다, 왜, 로, 나오다, 제, 심기, 불편하다]"
5,7898805,음악이 주가 된 최고의 음악영화,1,"[음악, 주가, 되다, 최고, 음악, 영화]"


In [32]:
negative_words = np.hstack(train_data[train_data['label'] == 0]['tokenzied'].values)
positive_words = np.hstack(train_data[train_data['label'] == 1]['tokenzied'].values)

In [36]:
negative_words_cnt = Counter(negative_words)
negative_words_cnt.most_common(20)

[('영화', 24409),
 ('보다', 17798),
 ('도', 12547),
 ('없다', 11111),
 ('들', 9451),
 ('만', 5488),
 ('다', 5481),
 ('이다', 5149),
 ('아니다', 4951),
 ('너무', 4850),
 ('있다', 4791),
 ('로', 4611),
 ('점', 4330),
 ('진짜', 4306),
 ('만들다', 4305),
 ('같다', 4123),
 ('아깝다', 4106),
 ('되다', 3977),
 ('않다', 3861),
 ('나오다', 3811)]

In [37]:
positive_words_cnt = Counter(positive_words)
positive_words_cnt.most_common(20)

[('영화', 25958),
 ('보다', 23274),
 ('좋다', 8719),
 ('들', 8694),
 ('도', 8532),
 ('이다', 8047),
 ('있다', 7638),
 ('재밌다', 7430),
 ('정말', 6445),
 ('너무', 6274),
 ('적', 5350),
 ('최고', 5320),
 ('되다', 4875),
 ('다', 4624),
 ('없다', 4461),
 ('같다', 4220),
 ('과', 4009),
 ('진짜', 4006),
 ('않다', 3861),
 ('감동', 3825)]

In [38]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [43]:
len(tokenizer.word_index)

43758

In [41]:
# 긍정 리뷰 (평균 길이: 10.9)
positive_text_length = train_data[train_data['label'] == 1]['tokenzied'].map(lambda x : len(x))
np.mean(positive_text_length)

10.952052522460262

In [42]:
# 부정 리뷰 (평균 길이: 10.9)
negative_text_length = train_data[train_data['label'] == 0]['tokenzied'].map(lambda x : len(x))
np.mean(negative_text_length)

11.466054242021823

In [ ]:
# 리뷰에 사용된 단어가 적을 경우 제외 
